# Identify & Track Marine Heatwaves on _Unstructured Grid_ using `spot_the_blOb`

## Processing Steps:
1. Fill spatial holes in the binary data, using `dask_image.ndmorph` -- up to `R_fill` cells in radius.
2. Fill gaps in time -- permitting up to `T_fill` missing time slices, while keeping the same blob ID.
3. Filter out small objects -- area less than the bottom `area_filter_quartile` of the size distribution of objects.
4. Identify objects in the binary data, using `dask_image.ndmeasure`.
5. Connect objects across time, applying the following criteria for splitting, merging, and persistence:
    - Connected Blobs must overlap by at least fraction `overlap_threshold` of the smaller blob.
    - Merged Blobs retain their original ID, but partition the child blob based on the parent of the _nearest-neighbour_ cell. 
6. Cluster and reduce the final object ID graph using `scipy.sparse.csgraph.connected_components`.
7. Map the tracked objects into ID-time space for convenient analysis.

N.B.: Exploits parallelised `dask` operations with optimised chunking using `flox` for memory efficiency and speed \
N.N.B.: This example using 40 years of _daily_ outputs at 5km resolution on an Unstructured Grid (15 million cells) using 32 cores takes 
- Full Split/Merge Thresholding & Merge Tracking:  ~40 minutes

In [1]:
import xarray as xr
import dask
from getpass import getuser
from pathlib import Path

import spot_the_blOb as blob
import spot_the_blOb.helper as hpc

In [2]:
# Start Dask Cluster
client = hpc.StartLocalCluster(n_workers=32, n_threads=2)

Memory per Worker: 7.86 GB
Hostname is  l10746
Forward Port = l10746:8787
Dashboard Link: localhost:8787/status


In [3]:
# Load Pre-processed Data (cf. `01_preprocess_extremes.ipynb`)

file_name = Path('/scratch') / getuser()[0] / getuser() / 'mhws' / 'extreme_events_binary_unstruct.zarr'
chunk_size = {'time': 4, 'ncells': -1}
ds = xr.open_zarr(str(file_name), chunks={}).isel(time=slice(0,32)).chunk(chunk_size)

In [4]:
# Tracking Parameters

drop_area_quartile = 0.8  # Remove the smallest 80% of the identified blobs
hole_filling_radius = 32   # Fill small holes with radius < 8 elements
time_gap_fill = 2         # Allow gaps of 2 days and still continue the blob tracking with the same ID
allow_merging = True      # Allow blobs to split/merge. Keeps track of merge events & unique IDs.
overlap_threshold = 0.5   # Overlap threshold for merging blobs. If overlap < threshold, blobs keep independent IDs.
nn_partitioning = True    # Use new NN method to partition merged children blobs. If False, reverts to old method of Di Sun et al. 2023...

In [5]:
# SpOt & Track the Blobs & Merger Events

tracker = blob.Spotter(ds.extreme_events, ds.mask, R_fill=hole_filling_radius, T_fill = time_gap_fill, area_filter_quartile=drop_area_quartile, 
                       allow_merging=allow_merging, overlap_threshold=overlap_threshold, nn_partitioning=nn_partitioning, 
                       xdim='ncells',                 # Need to tell spot_the_blOb the new Unstructured dimension
                       unstructured_grid=True,        # Use Unstructured Grid
                       neighbours=ds.neighbours,      # Connectivity array for the Unstructured Grid
                       cell_areas=ds.cell_areas,      # Cell areas for each Unstructured Grid cell
                       debug=0)                       # Choose debugging level

blobs = tracker.run(return_merges=False)

blobs

Constructing the Sparse Dilation Matrix...
Finished filling spatio-temporal holes.
Finished filtering small blobs.
Finished blob identification.
Finished calculating blob properties.
Finished finding overlapping blobs.
Processing iteration 1 with 6 merging blobs...
Processing iteration 2 with 9 merging blobs...
Processing iteration 3 with 6 merging blobs...
Finished splitting and merging blobs.
Finished clustering and renaming blobs.


Finished tracking blobs.


Tracking Statistics:
   Binary Hobday to Processed Area Fraction: 0.7365573740794455
   Total Object Area IDed (cells): 32497142
   Number of Initial Pre-Filtered Blobs: 1653
   Area Cutoff Threshold (cells): 21838.800000000014
   Accepted Area Fraction: 0.8044298480155578
   Total Blobs Tracked: 34
   Total Merging Events: 32
   Multi-Parent Merging Events: 0


<xarray.Dataset> Size: 2GB
Dimensions:       (ncells: 14886338, time: 32, ID: 34, component: 2,
                   sibling_ID: 2)
Coordinates:
    lat           (ncells) float64 119MB dask.array<chunksize=(14886338,), meta=np.ndarray>
    lon           (ncells) float64 119MB dask.array<chunksize=(14886338,), meta=np.ndarray>
  * time          (time) datetime64[ns] 256B 2002-04-11T23:59:00 ... 2002-05-...
  * ID            (ID) int32 136B 1 2 3 4 5 6 7 8 9 ... 27 28 29 30 31 32 33 34
Dimensions without coordinates: ncells, component, sibling_ID
Data variables:
    ID_field      (time, ncells) int32 2GB dask.array<chunksize=(4, 14886338), meta=np.ndarray>
    global_ID     (time, ID) int32 4kB 1 2 3 4 5 6 7 8 ... 328 0 323 0 0 329 331
    area          (time, ID) float32 4kB dask.array<chunksize=(10, 34), meta=np.ndarray>
    centroid      (component, time, ID) float32 9kB dask.array<chunksize=(1, 10, 34), meta=np.ndarray>
    presence      (time, ID) bool 1kB True True True True ... False True True
    time_start    (ID) datetime64[ns] 272B 2002-04-11T23:59:00 ... 2002-05-11...
    time_end      (ID) datetime64[ns] 272B 2002-05-01T23:59:00 ... 2002-05-12...
    merge_ledger  (time, ID, sibling_ID) int32 9kB dask.array<chunksize=(4, 34, 2), meta=np.ndarray>
Attributes: (12/13)
    allow_merging:               1
    N_blobs_prefiltered:         1653
    N_blobs_final:               34
    R_fill:                      32
    T_fill:                      2
    area_filter_quartile:        0.8
    ...                          ...
    accepted_area_fraction:      0.8044298480155578
    preprocessed_area_fraction:  0.7365573740794455
    overlap_threshold:           0.5
    nn_partitioning:             1
    total_merges:                32
    multi_parent_merges:         0

In [ ]:
# Save Tracked Blobs to `zarr` for more efficient parallel I/O

file_name = Path('/scratch') / getuser()[0] / getuser() / 'mhws' / 'MHWs_tracked_unstruct.zarr'
blobs.to_zarr(file_name, mode='w')